In [1]:
#= 📦 Importy i załadowanie danych =#
using JLD2
using Printf
using Statistics
using LinearAlgebra

include("src/AD.jl")
include("src/NN.jl")

Main.NN

In [2]:
#= 🔄 Przygotowanie danych =#
file = load("data/imdb_dataset_prepared.jld2")
X_train = Int.(file["X_train"])
y_train = vec(Float32.(file["y_train"]))
X_test  = Int.(file["X_test"])
y_test  = vec(Float32.(file["y_test"]))
embeddings = file["embeddings"]
vocab = file["vocab"]
embedding_dim = size(embeddings, 1)

50

In [3]:
dataset = NN.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

Main.NN.DataLoader((Float32[6391.0 286.0 … 11382.0 8311.0; 143.0 286.0 … 4254.0 2891.0; … ; 12849.0 12849.0 … 12849.0 12849.0; 12849.0 12849.0 … 12849.0 12849.0], Float32[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]), 64, true, [12738, 23640, 23530, 26716, 33576, 19342, 18418, 19608, 32789, 33893  …  16657, 14605, 28255, 36065, 12817, 26706, 22524, 5049, 20028, 13555])

In [4]:
model = NN.Chain(
    x -> Int.(x),
    NN.Embedding(length(vocab), embedding_dim, embeddings),
    x -> permutedims(x, (2, 1, 3)),
    NN.Conv1D(3, embedding_dim, 8, NN.relu),
    NN.MaxPool1D(8),
    NN.flatten,
    NN.Dense(128, 1, NN.sigmoid)
)

Main.NN.Chain(Any[var"#11#13"(), Main.NN.Embedding(12849, 50, Float32[0.90951 -0.58014 … 0.014323 0.0; -0.20702 -1.1316 … -0.74624 0.0; … ; -0.60396 0.1722 … -0.079873 0.0; -0.31136 1.2622 … -0.89164 0.0], Int64[]), var"#12#14"(), Main.NN.Conv1D(3, 50, 8, Float32[-0.011274268 0.19492935 … -0.09591613 0.0094998125; -0.15421052 -0.055225443 … -0.1638502 -0.12043111; -0.031841464 0.07107344 … 0.031222705 -0.2271448;;; -0.0811258 -0.023573885 … 0.11729122 -0.122308366; -0.08746908 0.10450979 … 0.08103245 0.16720204; -0.024774123 -0.051743537 … 0.046120048 -0.17081676;;; -0.12733173 -0.10502957 … -0.052491613 -0.15518901; 0.22661838 -0.13931379 … 0.07032786 0.04317539; -0.10640272 0.107108474 … -0.0061525144 -0.04017419;;; 0.022279993 0.068236105 … -0.13720897 -0.0700028; -0.093604416 -0.09348902 … -0.0799094 0.12840435; -0.0683449 0.25528893 … -0.021777505 -0.020534312;;; -0.031141192 -0.0026964345 … -0.1623106 -0.06560745; -0.19341394 0.1544467 … 0.05053372 -0.30262625; 0.125327 0.2662838

In [5]:
accuracy(m, x, y) = mean((vec(m(x)) .> 0.5) .== (y .> 0.5))

accuracy (generic function with 1 method)

In [6]:
#= 🚀 Trening modelu =#
opt = NN.Adam(Float32(0.001))
epochs = 5
NN.train_model(model, dataset, X_test, y_test, opt, epochs)

Epoch: 1 (92.06s) 	Train: (l: 0.59, a: 0.67) 	Test: (l: 0.49, a: 0.77) 	Memory allocated: 24761.250 MB
Epoch: 2 (72.32s) 	Train: (l: 0.48, a: 0.78) 	Test: (l: 0.46, a: 0.79) 	Memory allocated: 24374.366 MB
Epoch: 3 (71.59s) 	Train: (l: 0.45, a: 0.79) 	Test: (l: 0.44, a: 0.80) 	Memory allocated: 24382.346 MB
Epoch: 4 (69.79s) 	Train: (l: 0.44, a: 0.80) 	Test: (l: 0.43, a: 0.81) 	Memory allocated: 24386.671 MB
Epoch: 5 (71.58s) 	Train: (l: 0.42, a: 0.81) 	Test: (l: 0.42, a: 0.82) 	Memory allocated: 24389.746 MB
